# Try out how to implement GDO in Python

As a conclusion, Python's RDD interface is way more earier to use than Scala's RDD interface. Therefore there is no need to even implemt the GDO, the Python RDD interface is convience enought.

In [1]:
a=app.createDF("id:String;dt:Integer", """
    1,1;
    1,2;
    1,5;
    1,8""")

### The old fasion way
Refer column as index of Row

In [7]:
r=a.rdd.groupBy(lambda r: r[0]).collect()

Row can be converted to Python dictionary by `asDict()`

In [22]:
for i in r: print [ (x.asDict()['id'], x.asDict()['dt']) for x in i[1] ]

[(u'1', 1), (u'1', 2), (u'1', 5), (u'1', 8)]


### The `DataSet` way
Refer columns using the dot on Row

In [23]:
for i in r: print [ x.id for x in i[1] ]

[u'1', u'1', u'1', u'1']


In [55]:
r2=a.rdd.groupBy(lambda r: r.id).map(lambda (k, v): [x.id for x in v]).collect()

In [56]:
r2

[[u'1', u'1', u'1', u'1']]

Create a simple function which maintain a state and output with the state column

In [57]:
from pyspark.sql import Row
def addCnts(v):
    res = []
    cnt = 0
    for r in v:
        newr=r.asDict()
        newr['cnt'] = cnt
        res.append(Row(**newr))
        cnt += 1
    return res

It is very easy to execute the RE function

In [61]:
r3=a.rdd.groupBy(lambda r: r.id).flatMap(lambda (k, v): addCnts(v)).toDF()

In [62]:
r3.collect()

[Row(cnt=0, dt=1, id=u'1'),
 Row(cnt=1, dt=2, id=u'1'),
 Row(cnt=2, dt=5, id=u'1'),
 Row(cnt=3, dt=8, id=u'1')]